In [88]:
from lark import Lark

In [100]:
grammar = r"""
start : expr
atom_expr : var | number
number : NUMBER
?expr : atom_expr
     | op_expr
     | paren_expr
     | brace_expr
     | brack_expr
op_expr : 
     | expr binary_op expr
     | left_unary_op expr
     | expr right_unary_op
left_unary_op: "-" | "\\neg" | "\\#"
right_unary_op: "+" | "\\#" | "!"
paren_expr: "(" expr ")"
brace_expr: "{" expr "}"
brack_expr: "[" expr "]"
binary_op : "+" | "-" | "*" | "\cdot"
div_expr : frac brace_expr brace_expr
         | (paren_expr | atom_expr) "/" (paren_expr | atom_expr) 
frac: "\\frac" | "\\dfrac"
var : var_atom [ script ]
var_atom : LATIN | ALPHA | BETA | GAMMA | DELTA | EPSILON | ZETA | ETA 
         | THETA | VARTHETA | IOTA | KAPPA | LAMBDA | MU | NU | XI | PI
         | RHO | VARRHO | SIGMA | TAU | UPSILON | PHI | VARPHI | CHI 
         | PSI | OMEGA
LATIN : /[a-z]/i
ALPHA : /\\alpha/i
BETA : /\\beta/i
GAMMA : /\\gamma/i
DELTA : /\\delta/i
EPSILON : /\\epsilon/i
ZETA : /\\zeta/i
ETA : /\\eta/i
THETA : /\\theta/i
VARTHETA : /\\vartheta/i
IOTA : /\\iota/i
KAPPA : /\\kappa/i
LAMBDA : /\\lambda/i
MU : /\\mu/i
NU : /\\nu/i
XI : /\\xi/i
PI : /\\pi/i
RHO : /\\rho/i
VARRHO : /\\varrho/i
SIGMA : /\\sigma/i
TAU : /\\tau/i
UPSILON : /\\upsilon/i
PHI : /\\phi/i
VARPHI : /\\varphi/i
CHI : /\\chi/i
PSI : /\\psi/i
OMEGA : /\\omega/i
?atom : var_atom    -> var
      | ANY
script : subscript [supscript]
       | supscript [subscript]
subscript : "_" (brace_expr | atom)
supscript : "^" (brace_expr | atom)
ANY : /./
ANY_STR : /./+
%import common.NUMBER
%import common.WS
%ignore WS
"""

In [102]:
parser = Lark(grammar, debug=True)
tree = parser.parse(r"\frac {a} {b}")
print(tree.pretty())

UnexpectedCharacters: No terminal matches '\' in the current parser context, at line 1 col 1

\frac {a} {b}
^
Expected one of: 
	* EPSILON
	* ZETA
	* SIGMA
	* PI
	* VARPHI
	* STAR
	* GAMMA
	* DELTA
	* BETA
	* TAU
	* THETA
	* CHI
	* LBRACE
	* IOTA
	* XI
	* VARRHO
	* PSI
	* LPAR
	* BANG
	* MINUS
	* UPSILON
	* VARTHETA
	* ALPHA
	* PLUS
	* LAMBDA
	* ETA
	* OMEGA
	* MU
	* KAPPA
	* __ANON_0
	* LSQB
	* NU
	* PHI
	* RHO
	* __ANON_1
	* LATIN
	* __ANON_2
	* NUMBER
